# Anxiety/Confidence Analysis

Steps:
1. Using seed words for the anxiety class and confidence class, use word2vec to find the 1000 most "anxious" and 1000 most "confident" words
2. For each class of documents (either topic, province, or topic/province), calculate the proportion of anxious words used in that class relative to the proportion of anxious words used in total
3. cassssh

In [1]:
from utils import DTYPE, PARSE_DATES, PROV_CONSOLIDATION, CONSOLIDATED_PROVINCES, CONVERTERS, ANCHOR_NAMES, PROVINCE_COLOR_MAP
from tqdm.auto import tqdm
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import glob
tqdm.pandas()


prov_map = lambda x : x if x not in PROV_CONSOLIDATION else PROV_CONSOLIDATION[x]

total_df = pd.read_csv("../data/processed_data/total_tweet_dataset.csv",header=0,dtype=DTYPE,converters=CONVERTERS,parse_dates=PARSE_DATES)
total_df = total_df.set_index("id").sort_values("created_at")[~total_df.index.duplicated()]

total_df["created_at"] = total_df["created_at"].dt.to_period("D").dt.to_timestamp('s')
total_df["province"] = total_df["province"].apply(prov_map)
total_df = total_df[total_df.clean_text.notnull()]
total_df["province"] = total_df["province"].apply(prov_map)
total_df = total_df[total_df["province"].isin(CONSOLIDATED_PROVINCES)]
print(len(total_df))

/Users/cameron/.local/share/virtualenvs/COVID-Child-Care-Twitter-OW82RhPk/lib/python3.7/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


402217


,created_at,screen_name,source,clean_text,original_text,is_retweet,favorite_count,retweet_count,city,province,longitude,latitude,cluster,cluster_name
id,,,,,,,,,,,,,,
1228700383482126337,2020-02-15,Dale32497473,Twitter for Android,tyrant behaves people afraid drop kid school s...,Only a tyrant behaves this way. When people ar...,True,0,0,Quebec,Quebec,-71.21454,46.81228,7,Overflow
1228701333127036934,2020-02-15,DanWKearney,Twitter for iPhone,beachglassfan shawnwilson1975 bluejay19758259 ...,@BeachGlassFan @ShawnWilson1975 @BlueJay197582...,False,1,1,Halifax,Atlantic Province,-63.57239,44.64533,6,School Closures
1228703017807032321,2020-02-15,DanWKearney,Twitter for iPhone,shawnwilson1975 bluejay19758259 itsfortodat hr...,@ShawnWilson1975 @BlueJay19758259 @itsfortodat...,False,0,0,Halifax,Atlantic Province,-63.57239,44.64533,6,School Closures
1228710813596356608,2020-02-15,TwistedEgo2026,Twitter for iPhone,work hard give child best child worry die hand...,WP: I work hard to give my children the best I...,True,0,0,Windsor,Ontario,-83.01654,42.30008,3,Remote Work
1228713336788811776,2020-02-15,NorthShoreNews,dlvr.it,school district hope open childcare centre tea...,#WestVan school district hopes to open childca...,False,41,11,Vancouver,British Columbia,-123.11934,49.24966,2,Childcare


## Bootstrapping

start with seed words for the anxious and confident classes

> We retain the 1,000 terms that are semantically closest, on average, to the anxiety seed words, and the 1,000 terms closest to the confidence seed words.

use word2vec and cosine similarity to build a big vocab of anxious/confident words

In [88]:
from gensim.models import KeyedVectors
import gensim.downloader as api

# # word_vectors = api.load("glove-wiki-gigaword-100")
# # word_vectors.save("../models/glove.kv")
# word_vectors = KeyedVectors.load("../models/glove.kv", mmap='r')
# anxious_seed = ["risk", "threat", "concerned", "doubt", "worry", "fear", "danger", "tension", "stress", "anxious", "upset", "alarming", "worried", "hazard", "uncertain", "scare", "unknown", "alarm", "tense", "anxiety", "distress", "nervous", "risky", "troubled", "threatening", "panic", "fearful", "frighten", "unrest", "doubtful"]
# confident_seed = ["ease", "protect", "sure", "security", "safety", "protection", "confidence", "safe", "trust", "hope", "guarantee", "assurance", "certainty", "secure", "confident", "known", "guaranteed", "reassure", "predictable", "quiet", "hopeful", "optimistic", "bold", "convinced", "optimism", "content", "reassurance", "calm", "faithful", "comfort"]
# def seed_similarity(word,seed):
#     return np.array([word_vectors.similarity(word,seed_word) for seed_word in seed]).mean() 
# vocab = pd.DataFrame({"word":word_vectors.vocab.keys()})
# vocab["anxious_similarity"] = vocab["word"].progress_apply(lambda x : seed_similarity(x,anxious_seed))
# anxious_values = vocab.sort_values("anxious_similarity",ascending=False).head(1000).to_csv("../data/external_datasets/anxious_words.csv")
# vocab["confident_similarity"] = vocab["word"].progress_apply(lambda x : seed_similarity(x,confident_seed))
# confident_values = vocab.sort_values("confident_similarity",ascending=False).head(1000).to_csv("../data/external_datasets/confident_words.csv")


In [2]:
from kaleido.scopes.plotly import PlotlyScope
scope = PlotlyScope()
vis_args = {
    "template": "simple_white",
    "font":{"size": 23},
    "width": 1000
}